In [1]:
import os
from dotenv import load_dotenv, find_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2, openai_api_key=OPENAI_API_KEY)

In [13]:
chat.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: 'Hello, how are you?'"
        )
    ]
).content

'Bonjour, comment vas-tu?'

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat

In [5]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: 'Hello, how are you?'"
            ),
            AIMessage(
                content="Bonjour, comment ça va?",
            ),
            HumanMessage(content="What did you just say?"),
        ]
    }
).content

'I said, "Hello, how are you?" in French.'

In [6]:
from langchain.memory import ChatMessageHistory

chat_hist = ChatMessageHistory()
chat_hist.add_user_message(
    "Translate this sentence from English to French: 'Hello, how are you?'"
)
chat_hist.add_ai_message("Bonjour, comment ça va?")
chat_hist.add_user_message("What did you just say?")
chat_hist.messages

[HumanMessage(content="Translate this sentence from English to French: 'Hello, how are you?'"),
 AIMessage(content='Bonjour, comment ça va?'),
 HumanMessage(content='What did you just say?')]

In [7]:
chain.invoke({"messages": chat_hist.messages}).content

'I said, "Hello, how are you?" in French.'

## Retrievers

Create a loader to pull data from a webpage:

In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("http://docs.smith.langchain.com/overview")
data = loader.load()

Split the resulting text into smaller chunks:

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

Embed those chunks and store in a vector database:

In [10]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

Create a retriever from the initialized `vectorstore`:

In [11]:
retriever = vectorstore.as_retriever(k=4)
docs = retriever.invoke("How can langsmith help with testing?")
docs

[Document(page_content='Get started with LangSmith | 🦜️🛠️ LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'http://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith'}),
 Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith\u200bPythonTypeScriptpip install -U langsmithyarn add langchain', m

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

chat = ChatOpenAI(
    model='gpt-4o',
    
)